In [14]:
import pandas as pd # type: ignore
df = pd.read_csv("admin-data.csv")
df.head()

,Id,Brand locations,Name,Name ar,Logo,Category,Sub category,Business category,Sub business category,Category v2,...,Business category v3,Sub business category v3,Website,International brand,Secondary type,V1,Archived,Recurring,Created at,Updated at
0,19742,SAU,AKxy Clothing Company,NaN,brand_logos/_AKxy_Clothing_Company_Saudi Arabi...,Shopping,Clothing & Accessories,NaN,NaN,Shopping,...,NaN,NaN,https://akclothingco.net,False,brand,33369.0,False,False,2025-04-07 09:16:20 UTC,2025-04-28 05:45:40 UTC
1,13815,KSA,AWS Lambda,NaN,brand_logos/_AWS_Lambda.png,Services,Other Services,NaN,NaN,Services,...,NaN,NaN,NaN,False,brand,NaN,False,False,2025-04-07 09:14:59 UTC,2025-04-07 09:14:59 UTC
2,19999,UAE,Al Khaleej Drug Store,NaN,brand_logos/_Al_Khaleej_Drug_Store_AE.png,Wellness,Pharmacies & Supplements,NaN,NaN,Wellness,...,NaN,NaN,https://www.khaleejdrugstore.ae/#home,False,brand,26107.0,False,False,2025-04-07 09:16:20 UTC,2025-04-07 09:16:20 UTC
3,14859,KSA,Al Khoory Sky Garden Hotel,فندق الخوري سكاي جاردن,brand_logos/_Al_Khoory_Sky_Garden_Hotel_AE.png,Travel,Hotels & Accomodation,NaN,NaN,Travel,...,NaN,NaN,https://www.alkhooryhotels.com/en/,False,brand,NaN,False,False,2025-04-07 09:14:59 UTC,2025-04-07 09:14:59 UTC
4,27368,BHR,Aldaaysi Market,NaN,brand_logos/_Aldaaysi_Market_BH.png,Groceries,Supermarkets & Hypermarkets,NaN,NaN,Groceries,...,NaN,NaN,https://www.instagram.com/aldaaysi/,False,brand,NaN,False,False,2025-04-07 09:17:16 UTC,2025-04-07 09:17:16 UTC


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27838 entries, 0 to 27837
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        27838 non-null  int64  
 1   Brand locations           27838 non-null  object 
 2   Name                      27838 non-null  object 
 3   Name ar                   18288 non-null  object 
 4   Logo                      22702 non-null  object 
 5   Category                  25869 non-null  object 
 6   Sub category              25869 non-null  object 
 7   Business category         0 non-null      float64
 8   Sub business category     0 non-null      float64
 9   Category v2               25870 non-null  object 
 10  Sub category v2           25870 non-null  object 
 11  Category v3               25878 non-null  object 
 12  Sub category v3           25878 non-null  object 
 13  Business category v2      0 non-null      float64
 14  Sub bu

In [16]:
new_df = df[["Name", "Website"]]
new_df

,Name,Website
0,AKxy Clothing Company,https://akclothingco.net
1,AWS Lambda,NaN
2,Al Khaleej Drug Store,https://www.khaleejdrugstore.ae/#home
3,Al Khoory Sky Garden Hotel,https://www.alkhooryhotels.com/en/
4,Aldaaysi Market,https://www.instagram.com/aldaaysi/
...,...,...
27833,‎Nabaa Al kawthar,NaN
27834,‎Rabie Lebnan Restaurant,https://www.facebook.com/LebaneseHouseRestaura...
27835,‎The Garden Medical Center,https://thegardenmc.com/promotions
27836,‏REEF Perfumes,https://reefperfumes.com/ar


In [17]:
# number of brands is 27,838 Brands

numberOfBrands = new_df["Name"].count()
numberOfBrands

np.int64(27838)

In [18]:
# number of brands with a website link is 20,868 Brands

numberOfBrandsWithAWebsiteLink = new_df["Website"].count()
numberOfBrandsWithAWebsiteLink

np.int64(20868)

In [19]:
# Brands with no website link is 6,970 Brands

brandsWithNoWebsiteLink = new_df["Name"].count() - new_df["Website"].count()
brandsWithNoWebsiteLink

np.int64(6970)

In [20]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

TIMEOUT = 5           # Increased timeout for slow sites
MAX_WORKERS = 100     # Number of threads

def is_link_working(url):
    if not isinstance(url, str) or not url.startswith("http"):
        return False
    try:
        headers = {
            'User-Agent': (
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                'AppleWebKit/537.36 (KHTML, like Gecko) '
                'Chrome/120.0.0.0 Safari/537.36'
            )
        }
        resp = requests.get(url, timeout=TIMEOUT, allow_redirects=True, headers=headers)
        # Accept any 2xx or 3xx status code as "working"
        return 200 <= resp.status_code < 400
    except Exception as e:
        # Uncomment for debugging:
        # print(f"Error checking {url}: {e}")
        return False

def check_links_parallel(df, url_column='Website', max_workers=MAX_WORKERS):
    results = [None] * len(df)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {
            executor.submit(is_link_working, df.iloc[i][url_column]): i
            for i in range(len(df))
        }
        for future in tqdm(as_completed(future_to_idx), total=len(future_to_idx), desc="🔗 Checking links"):
            idx = future_to_idx[future]
            try:
                results[idx] = future.result()
            except Exception:
                results[idx] = False
    df['link_working'] = results
    return df

if __name__ == "__main__":
    new_df = pd.read_csv("admin-data.csv")
    new_df = new_df.dropna(subset=['Name', 'Website'])
    checked_df = check_links_parallel(new_df)
    working = checked_df['link_working'].sum()
    total = len(checked_df)
    print(f"\n✅ Working: {working}")
    print(f"❌ Not working: {total - working}")
    checked_df[['Name', 'Website', 'link_working']].to_excel("URL_CHECKER_RESULTS/admin-data_URL_CHECKER_RESULTS.xlsx", index=False)

🔗 Checking links: 100%|██████████| 20868/20868 [05:26<00:00, 63.97it/s] 



✅ Working: 14852
❌ Not working: 6016


In [21]:
# Performance 

Accuracy = (working / total) * 100

print(f"Accuracy: {Accuracy:.2f}%")

Accuracy: 71.17%


before: 56.85%

after: 73.33%